In [ ]:
# 라이브러리 설치
!pip install requests
!pip install openpyxl
!pip install pyperclip
!pip install selenium
!pip install pandas
!apt install chromium-chromedriver 

In [ ]:
# 라이브러리 임포트
import requests
from openpyxl.workbook import Workbook
import time
import pyperclip
import re
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

# 추가한 라이브러리
from bs4 import BeautifulSoup 

In [ ]:
# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument("--start-fullscreen")
options.add_argument("window-size=1920x1080") 
options.add_argument("disable-gpu") 
options.add_argument("no-sandbox") 
options.add_argument("--disable-dev-shm-usage")

In [ ]:
# 영화 상세페이지 체크 함수
def check_exists_by_xpath(ch_driver, xpath):
    try:
        ch_driver.find_element_by_xpath(xpath) 
    except NoSuchElementException: 
        return False
    return True

In [ ]:
# 변수 생성
movie_collect = {}
title_infos = [] # 제목
content_infos = [] # 줄거리
genre_infos = [] # 장르
score_infos = [] # 평점
date_infos = [] # 개봉일

In [ ]:
# 변수 초기화
title_infos.clear()
content_infos.clear()
genre_infos.clear()
score_infos.clear()
date_infos.clear()

In [ ]:
# 주소 접속
driver = webdriver.Chrome("chromedriver", options=options)
url = "https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&qvt=0&query=%EC%97%AD%EB%8C%80%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84"
driver.get(url) 
driver.get_screenshot_as_file('create_headless_driver.png')
time.sleep(2)

In [ ]:
# 영화 상세보기 페이지 이동
# 총 13페이지를 반복하기 위한 for문\
# 총 페이지수 변경 시 14 대신 마지막 페이지 수 입력
for i in range(1, 14): 
  page_cnt = str(i)

  # 8개의 영화를 긁어오기 위한 변수
  two_range = 9 

  # 영화 정보 수집하기 위한 for문         
  for j in range(1, two_range): 
    try:
        movie_cnt = str(j)

        # 알라딘 영화 구분을 위한 변수
        temp_title = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/div[2]/div/div[1]/div[2]/div[' + page_cnt + ']/ul/li[' + movie_cnt + ']/div[2]/a/div/strong').text
        print(temp_title)

        # 영화의 이미지를 클릭
        movie = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/div[2]/div/div[1]/div[2]/div[' + page_cnt + ']/ul/li[' + movie_cnt + ']/div[1]/a/span[1]').click() # 클릭을 실행하는 함수
        time.sleep(4)
  
        # //*[@id="main_pack"]/div[3]/div[1]/div[1]/h2/a/strong <- 알라딘 div[3]
        # //*[@id="main_pack"]/div[2]/div[1]/div[1]/h2/a/strong <- 다른 영화 div[2]
        basic_num = '2'
        if temp_title == '알라딘':
            basic_num = '3'

        # 영화 제목
        try: 
            movie_title = driver.find_element_by_xpath('//*[@id="main_pack"]/div[' + basic_num + ']/div[1]/div[1]/h2/a/strong').text
        except: 
            movie_title = temp_title

        # 평점
        if check_exists_by_xpath(driver, '//*[@id="main_pack"]/div[' + basic_num + ']/div[2]/div[1]/div[2]/div[2]/dl/div[3]/dd',): # 영화 상세보기 페이지에 평점 위치가 존재할 경우 위에서 정의한 함수에서 True 값을 받아 if문 수행
            movie_score = driver.find_element_by_xpath('//*[@id="main_pack"]/div[' + basic_num + ']/div[2]/div[1]/div[2]/div[2]/dl/div[3]/dd').text # 영화 상세보기 페이지에 평점 위치에 평점이 있을 경우 text를 추출하여 movie_score 변수에 담는다
        else: 
            movie_score = np.nan 

        # 장르
        # 액션 한국 118분 <- 국가, 상영시간 제거
        try:
            movie_genre = driver.find_element_by_xpath('//*[@id="main_pack"]/div[' + basic_num + ']/div[2]/div[1]/div[2]/div[2]/dl/div[1]/dd').text
            # 공백제거
            movie_genre = movie_genre.replace(',' , '').replace(' ','') 
            # 상영시간, 국가 제거
            movie_genre2 = re.sub(r'[0-9]+분', '', movie_genre).strip() 
            movie_genre3 = re.sub(r'(한국)|(미국)|(일본)|(캐나다)|(영국)|(스페인)|(중국)', '', movie_genre2)
            movie_genre3 = movie_genre3.strip() 

        except: 
            movie_genre = np.nan 

        # 줄거리 더보기 클릭
        try: 
            more_content = driver.find_element_by_xpath('//*[@id="main_pack"]/div[' + basic_num + ']/div[2]/div[1]/div[2]/div[2]/div/a').click()
        except: 
            more_content = np.nan 

        # 줄거리(더보기 페이지)
        try: 
            movie_content = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div[2]/div[2]/div/div[2]/p').text
            time.sleep(3) 
            driver.back() # 뒤로가기(줄거리 더보기 페이지 -> 영화 상세보기 페이지)
            time.sleep(2.5) 
        except:
            movie_content = np.nan 
        
            
        # 수집된 정보 출력
        print("=" *50 + "영화 데이터 수집 중입니다." + "="*50)
        print("수집한 제목: ", movie_title)
        print("수집한 장르: ", movie_genre3)
        print("수집한 평점: ", movie_score)
        print("수집한 줄거리: ", movie_content)
        print("="*120)

        # 생성된 변수에 append
        title_infos.append(movie_title)
        genre_infos.append(movie_genre3)
        score_infos.append(movie_score)
        content_infos.append(movie_content)

        driver.back() # 뒤로가기(영화 상세보기 페이지 -> 영화 목록 페이지)
        time.sleep(2)

        # 현재 영화가 포함된 페이지로 이동
        # ex) 4페이지로 이동하려면 '>' 버튼 3번 클릭 
        if i > 1: 
            for click_one in range(1, i): 
                print("----------------- 다음 페이지 클릭 >>>>> -----------------")
                next_page = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/div[2]/div/div[1]/div[3]/div/a[2]').click()
                time.sleep(0.7)
    except:
      print("<<<<<<<<<<<<<< 마지막 영화입니다 >>>>>>>>>>>>>>>") # # 마지막 페이지에서 출력못한 영화 n개 만큼 출력

  print("******* 이 페이지는 수집 끝! 다음 페이지로! >> *******")
  next_page = driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div/div[2]/div/div[1]/div[3]/div/a[2]').click()

driver.close()
print("======================== 역대흥행순위 TOP 100 영화 크롤링을 마쳤습니다.=========================================")


In [ ]:
# 각 컬럼에 데이터 저장    
movie_collect["제목"] = title_infos
movie_collect["장르"] = genre_infos
movie_collect["평점"] = score_infos
movie_collect["줄거리"] = content_infos

In [ ]:
# DataFrame으로 변환
df = pd.DataFrame({k: np.nan if not v else v for k, v in movie_collect.items()})

In [ ]:
# csv파일로 저장
df.to_csv("영화_TOP100_final.csv", index=False, encoding='utf-8-sig')